# Przetwarzanie Grafiki i Muzyki - laboratorium nr 4

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Zadanie 1 (1.5 punkta)
Proszę zaimplementować (ręcznie) konwersję z formatu RGB na YCbCr (https://en.wikipedia.org/wiki/YCbCr) i przetestować na wybranym przez siebie obrazku. Wzór:

$$
\begin{align}
Y &= 0.299 \cdot R + 0.587\cdot G + 0.114\cdot B \\
C_B &= 0.5 -0.169\cdot R - 0.331 \cdot G + 0.5\cdot B \\
C_R &= 0.5 + 0.5\cdot R - 0.419\cdot G - 0.081\cdot B,
\end{align}
$$

gdzie $R, G, B \in [0,1]$.

## Zadanie 2 (1.5 punkta)

Proszę zaimplementować ręcznie konwersję z formatu YCbCr na RGB (https://en.wikipedia.org/wiki/YCbCr). Przetestuj kod na obrazie wynikowym z poprzedniego zadania. Wzór:

$$
\begin{align}
    R &= Y + 1.402*(C_R - 0.5) \\
    G &= Y - 0.344*(C_B - 0.5)-0.714*(C_R - 0.5) \\
    B &= Y + 1.772*(C_B - 0.5)
\end{align}
$$

gdzie $Y, C_R, C_B \in [0, 1]$.

## Zadanie 3 (1 punkt)
Proszę zaimplementować konwersję z formatu RGB na HLS i wyświetlić składowe H, L oraz S. Tutaj można użyć gotowej metody z pakietu OpenCV (cv2.COLOR_BGR2HLS).

## Zadanie 4 (2 punkty)

Proszę stworzyć obrazek zawierający tylko zielone papryczki (analogicznie do przykładu z wykładu). Użyj obrazku "peppers.png".

HINT - zielone papryczki znajdują się tam, gdzie $R\in [20, 120]$ i $G \in [20, 160]$. R i G to natężenie składowych koloru w modelu RGB.

## Zadanie 5 (2 punkty)
Proszę dokonać detekcji skóry na zdjęciu "face.jpg", używając modelu HLS.

HINT - skóra znajduje się tam, gdzie spełniona jest następująca reguła:
$$
\left(S \geq 50\right) \land \left(0.5 < \frac{L}{S} < 3 \right) \land \left( \left(H \leq 14\right) \lor \left(H \geq 165\right) \right) 
$$

![alt](https://raw.githubusercontent.com/przem85/PGiM/master/img/face.jpg)

##  Zadanie 6 (2 punkty)
Proszę dokonać detekcję efektu czerwonych oczu na zdjęciu "face1.jpg".

HINT - czerwone oczy znajdują się tam, gdzie jest spełniona następująca reguła:
$$
\left( L \geq 64 \right) \land \left(S \geq 100\right) \land \left(0.5 < \frac{L}{S} < 1.5 \right) \land \left( \left(H \leq 7\right) \lor \left(H \geq 162\right) \right) 
$$

![alt](https://raw.githubusercontent.com/przem85/PGiM/master/img/face1.jpg)